In [ ]:
import numpy as np
from scipy.sparse import lil_matrix
from scipy.sparse import coo_matrix
from scipy.sparse import dok_matrix
import matplotlib.pyplot as plt
import torch.nn as nn
import sys
import math
import torch
import random

In [ ]:
class sparse_linnear(nn.Module):
    def __init__(self, dimensions):
        """
        :param dimensions: (tpl/ list) Dimensions of the neural net. (input, output)
        Example of three hidden layer with
        - input features
        - output features
        layers -->    [0,        1]
        ----------------------------------------
        dimensions =  (input features,     output features)
        """
        self.n_layers = len(dimensions)
        self.loss = None
        self.learning_rate = None
        self.momentum = None
        self.weight_decay = None
        self.epsilon = 20  # control the sparsity level as discussed in the paper
        self.zeta = 0.3  # the fraction of the weights removed
        self.droprate = 0  # dropout rate
        self.dimensions = dimensions
        # activations will be chosen from <activations>
        self.activations = ['Relu', 'Sigmoid', 'tanh']
        # Weights and biases are initiated by index. For a one hidden layer net you will have a w[1] and w[2]
        self.w = [] # weight matrix
        self.x = [] # adjacency matrix
        self.b = np.zeros(dimensions[1])
        # Here is the gradients that need to be backpropagated (I will write them in the future)
        
    def prob_connection(self, T, R, Theta, m, rj, theta_j, beta):
        # calculate the probability connections
        if T == 0:
            dij = {}
            for i in range(len(R)):
                d = hyperbolic_dist(R[i], rj, hyperbolic_angle(Theta[i], theta_j))
                dij[d] = i

            dist = sorted(dij.keys(), reverse = True)
            idx = []
            for i in range(m):
                idx.append(dij[dist[i]])

            return idx

        else:
            idx = []
            for i in range(len(R)):
                theta_ij = self.hyperbolic_angle(Theta[i], theta_j)
                hij = self.hyperbolic_dist(R[i], rj, theta_ij)
                radius_i = self.radius_of_Hyperbolic_disk(R[i], T, beta, i+2)
                prob_ij = 1/(1 + math.exp((hij - radius_i)/(2*T)))
                p = random.random()
                if prob_ij >= p:
                    idx.append(i)
            return idx
        
    def radius_of_Hyperbolic_disk(self, ri, T, beta, i):
        Ri = ri - 2*math.log(2*T*(1-math.exp(-(1-beta)*math.log(i)))
                             /(np.sin(T*np.pi) * m * (1-beta)))
        return Ri

    def hyperbolic_dist(self, ri, rj, theta_ij):
        hij = np.arccosh(np.cosh(ri) * np.cosh(rj) - np.sinh(ri) * np.sinh(rj) * np.cos(theta_ij))
        return hij

    def hyperbolic_angle(self, theta_i, theta_j):
        theta_ij = np.pi - abs(np.pi - abs(theta_i-theta_j))
        return theta_ij
    
    def createSparseWeights_with_nPSO_Gaussian(self, N, T, gamma, m, C):
        # generate an nPSO sparse weights mask
        beta = 1 / (gamma - 1)
        sigma = 2 * np.pi/(6 * C)
        p = 1/C
        R_1_2 = [] # two layers of nerouns
        Theta_1_2 = [] # the angle of nerouns reflect in the hyperbolic space
        # for bipartite network, two layers of neurons
        for k in range(2):
            R = []
            Theta = []
            for i in range(1,N+1):
                r = 2 * math.log(i) 
                # generate angle with gauss distribution
                mu = 2 / C  * np.pi * ((i-1) % C)
                theta = random.gauss(mu, sigma) %  (2 * np.pi)
                R.append(r)
                Theta.append(theta)
                if i > 1:
                    for j in range(i-1):
                        R[j] = beta * 2 * math.log(j + 1) + (1-beta)* 2 * math.log(i) 
            R_1_2.append(R)
            Theta_1_2.append(Theta)

        ax = plt.subplot(111, projection='polar')
        c = ax.scatter(Theta, R, cmap='cool', alpha=1)
        row = []
        col = []
        data1 = []
        data2 = []
        for i in range(len(R_1_2[0])):
            # idx is the connected links calculated by nPSO
            idx = self.prob_connection(T, R_1_2[1], Theta_1_2[1], m, R_1_2[0][i], Theta_1_2[0][i], beta)   
            for j in idx:
                row.append(i)
                col.append(j)
                # initialize the weight
                data1.append(np.float64(np.random.randn() / 10))
                data2.append(1)
        # initialize the sparse weight matrix
#         data1 = nn.Parameter((len(row)))
        weight_matrix = coo_matrix((data1, (row,col)),shape=(N,N))
        x = coo_matrix((data2, (row,col)), shape=(N,N))
        print("Create sparse matrix with ", weight_matrix.getnnz(), " connections and ",
                      (weight_matrix.getnnz() / (N * N)) * 100, "% density level")
        self.w = weight_matrix
        self.x = x.toarray()
        return weight_matrix.tocsr()
    
    
    def createSparseWeights_with_ER(self, epsilon, noRows, noCols):
        # generate an Erdos Renyi sparse weights mask
        # epsilon: control the sparsity level as discussed in the paper
        # noRows, noCols: the numbers of two layers of nerouns
        weights = lil_matrix((noRows, noCols))
        for i in range(epsilon * (noRows + noCols)):
            weights[np.random.randint(0, noRows), np.random.randint(0, noCols)] = np.float64(np.random.randn() / 10)
        print("Create sparse matrix with ", weights.getnnz(), " connections and ",
              (weights.getnnz() / (noRows * noCols)) * 100, "% density level")
        self.w = weights.tocsr()
        return weights.tocsr()
    
    

    def weightsEvolution_I(self):
#         this represents the core of the SET procedure. 
#         It removes the weights closest to zero in each layer and add new random weights

        values = np.sort(self.w.data)
        firstZeroPos = self.find_first_pos(values, 0)
        lastZeroPos = self.find_last_pos(values, 0)

        largestNegative = values[int((1 - self.zeta) * firstZeroPos)]
        smallestPositive = values[
            int(min(values.shape[0] - 1, lastZeroPos + self.zeta * (values.shape[0] - lastZeroPos)))]

        wlil = self.w.tolil()
        wdok = dok_matrix((self.dimensions[0], self.dimensions[1]), dtype="float64")


        # remove the weights closest to zero
        keepConnections = 0
        for ik, (row, data) in enumerate(zip(wlil.rows, wlil.data)):
            for jk, val in zip(row, data):
                if ((val < largestNegative) or (val > smallestPositive)):
                    wdok[ik, jk] = val

                    keepConnections += 1

        # add new random connections
        for kk in range(self.w.data.shape[0] - keepConnections):
            ik = np.random.randint(0, self.dimensions[0])
            jk = np.random.randint(0, self.dimensions[1])
            while (wdok[ik, jk] != 0):
                ik = np.random.randint(0, self.dimensions[0])
                jk = np.random.randint(0, self.dimensions[1])
            wdok[ik, jk] = np.random.randn() / 10
            
        self.w = wdok.tocsr()
        return wdok.tocsr()
    
    def weightsEvolution_II(self):
#       this represents the core of the CH2-L3 procedure. It removes the weights 
#       closest to zero in each layer and add new weights with CHA methods

        values = np.sort(self.w.data)
        firstZeroPos = self.find_first_pos(values, 0)
        lastZeroPos = self.find_last_pos(values, 0)

        largestNegative = values[int((1 - self.zeta) * firstZeroPos)]
        smallestPositive = values[
            int(min(values.shape[0] - 1, lastZeroPos + self.zeta * (values.shape[0] - lastZeroPos)))]

        wlil = self.w.tolil()
        wdok = dok_matrix((self.dimensions[0], self.dimensions[1]), dtype="float64")


        # remove the weights closest to zero
        keepConnections = 0
        for ik, (row, data) in enumerate(zip(wlil.rows, wlil.data)):
            for jk, val in zip(row, data):
                if ((val < largestNegative) or (val > smallestPositive)):
                    wdok[ik, jk] = val
                    keepConnections += 1
        
        # add new random connections with CH2_L3
        non_connect_nodes, sorted_index = self.sorted_non_connected_links_with_CH2_L3()
        weight_values = np.float64(np.random.randn(self.w.data.shape[0] - keepConnections, 1) / 10)
        self.w = wdok.tocoo()
        row, col = self.w.nonzero()
        data1 = self.w.data
        for i in range(len(weight_values)):
            row = np.append(row,non_connect_nodes[sorted_index[i]][0])
            col = np.append(col,non_connect_nodes[sorted_index[i]][1])
            data1 = np.append(data1, weight_values[i])
            self.x[non_connect_nodes[sorted_index[i]][0]][non_connect_nodes[sorted_index[i]][1]] = 1
        self.w = coo_matrix((data1, (row,col)),shape = (self.dimensions[0], self.dimensions[1]))

        return self.w
        
    def sorted_non_connected_links_with_CH2_L3(self):
        row_sum = np.reshape(self.x.sum(axis = 1), (100,-1))
        col_sum = np.reshape(self.x.sum(axis = 0), (100,-1))

        non_connect_nodes = find_zero_items(self.x)
        m = len(non_connect_nodes)
        A1 = []
        A2 = []
        for i in range(len(self.x)):
            A1.append(np.nonzero(self.x[i,:]))
            A2.append(np.nonzero(self.x[:,i]))
        CH2_L3_scores = []
        for i in range(m):
            u = non_connect_nodes[i][0]
            v = non_connect_nodes[i][1]
            Au = A2[v]
            Av = A1[u]
            x_Auv = self.x[Au[0]]
            x_Auv = x_Auv[:,Av[0]]

            a, b = np.nonzero(x_Auv)
            dict_a = get_item_nums(a)
            dict_b = get_item_nums(b)

            scores_CH2_L3 = 0
            for j in range(len(a)):
                i_num_x = dict_a[a[j]]
                i_num_y = dict_b[b[j]]
                idx_x = Au[0][a[j]]
                idx_y = Av[0][b[j]]
                deg_x = row_sum[idx_x]
                deg_y = col_sum[idx_y]
                e_num_x = deg_x - i_num_x
                e_num_y = deg_y - i_num_y
                scores_CH2_L3 += math.sqrt((1 + i_num_x)*(1 + i_num_y))/math.sqrt((1 + e_num_x)*(1 + e_num_y))
            CH2_L3_scores.append(scores_CH2_L3)
        dict_scores = {j:i for i, j in enumerate(CH2_L3_scores)}
        sorted_scores = sorted(list(dict_scores.keys()), reverse =True)
        index = []
        for i in sorted_scores:
            index.append(dict_scores[i])
        
        return non_connect_nodes, index
    
    def find_first_pos(self, array, value):
        idx = (np.abs(array - value)).argmin()
        return idx
    
    def find_last_pos(self, array, value):
        idx = (np.abs(array - value))[::-1].argmin()
        return array.shape[0] - idx

In [ ]:
def find_zero_items(A):
    non_connect_nodes = []
    for i in range(len(A)):
        for j in range(len(A)):
            if A[i,j] == 0:
                non_connect_nodes.append([i,j])
                
    return non_connect_nodes

def get_item_nums(arr):
    key = np.unique(arr)
    result = {}
    for k in key:
        mask = (arr == k)
        arr_new = arr[mask]
        v = arr_new.size
        result[k] = v
    return result

In [ ]:
model = sparse_linnear((100,100))
N = 100
T = 0.6
gamma = 3
m = 15
C = 4
w = model.createSparseWeights_with_nPSO_Gaussian(N, T, gamma, m, C)
x = model.x
# w = torch.Tensor(w.toarray())
# scipy_sparse_mat_to_torch_sparse_tensor(w)

In [ ]:
plt.imshow(w.toarray(), cmap='jet')
plt.show()

In [ ]:
torch.cuda.set_device(1)
w1 = torch.Tensor(w.toarray()).cuda()
w1.requires_grad_()
x = torch.Tensor(x).cuda()
input1 = torch.randn(100, 1).cuda()
input2 = torch.randn(100, 1).cuda()

In [ ]:
import datetime
criterion = torch.nn.MSELoss()
start = datetime.datetime.now()
for i in range(100000):
    w2 = w1 * x
    output = torch.mm(w2, input1)
    loss = criterion(output, input2)
    optimizer = torch.optim.SGD([w1], lr = 0.1)
    optimizer.zero_grad()   # reset gradient
    loss.backward()
    optimizer.step()  
    
end = datetime.datetime.now()
print("Start running time: %s" % (end-start))

In [ ]:
criterion = torch.nn.MSELoss()
start = datetime.datetime.now()
for i in range(100000):
#     w2 = w1 * x
    output = torch.mm(w1, input1)
    loss = criterion(output, input2)
    optimizer = torch.optim.SGD([w1], lr = 0.1)
    optimizer.zero_grad()   # reset gradient
    loss.backward()
    optimizer.step()  
    
end = datetime.datetime.now()
print("Start running time: %s" % (end-start))

In [ ]:
def scipy_sparse_mat_to_torch_sparse_tensor(sparse_mx):
    """
    将scipy的sparse matrix转换成torch的sparse tensor.
    """
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [ ]:
w = scipy_sparse_mat_to_torch_sparse_tensor(w)

w_values = w._values().requires_grad_()

In [ ]:
from torch_scatter import scatter

input1 = torch.randn(100, 1)
in_ = torch.empty(len(w_values))
input2 = torch.randn(100, 1)
output = torch.empty(len(w._indices()[0]))
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD([w_values], lr = 0.1)
for idx in range(len(w._indices()[0])):
    in_[idx] = input1[w._indices()[0][idx]]
start = datetime.datetime.now()
for i in range(100000):
    optimizer.zero_grad()   # reset gradient
#     for idx in range(len(w._indices()[0])):
#         output[idx] = input1[w._indices()[0][idx]]*w_values[idx]\
    
    output = in_ * w_values
    src = output
    index = w._indices()[1]
#     out = torch.zeros(100)
#     for idx in range(len(w._indices()[0])):
#         out[w._indices()[1][idx]] += src[idx]
    out = scatter(src, index, dim=0, reduce="sum")

    loss = criterion(out, input2)
    loss.backward()
    optimizer.step()
#     print(1)
    
end = datetime.datetime.now()
print("Start running time: %s" % (end-start))